# 01 Importing Libraries and Data for the task

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path= r'C:\Users\jphil\OneDrive\Documents\CareerFoundry Data Analytics\Data Immersion\Achievement 4\Data\Prepared Data'

In [5]:
ords_prods_merge=pd.read_pickle(os.path.join(path, 'ords_prods_merged.pkl'), compression='infer')

In [6]:
#finding the mean order number for the entire df
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


In [ ]:
#overall there doesn't seem to be much difference here compared to doing the same function for just the df subset
# some numbers are higher, some are lower

In [7]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,price_range_loc,busiest day,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,11.1,196,1,0,both,...,77.0,7.0,9.0,Mid-range product,Regularly busy,Regularly busy,Regularly busy,Average orders,10,New customer
1,2539329,1,1,2,8,11.1,14084,2,0,both,...,91.0,16.0,12.5,Mid-range product,Regularly busy,Regularly busy,Regularly busy,Average orders,10,New customer
2,2539329,1,1,2,8,11.1,12427,3,0,both,...,23.0,19.0,4.4,Low-range product,Regularly busy,Regularly busy,Regularly busy,Average orders,10,New customer
3,2539329,1,1,2,8,11.1,26088,4,0,both,...,23.0,19.0,4.7,Low-range product,Regularly busy,Regularly busy,Regularly busy,Average orders,10,New customer
4,2539329,1,1,2,8,11.1,26405,5,0,both,...,54.0,17.0,1.0,Low-range product,Regularly busy,Regularly busy,Regularly busy,Average orders,10,New customer


In [ ]:
#the next step is to create a loyalty flag for existing customers
#however, I did this step in the previous notebook that I used to follow along in the exercise
#here is the code I used to create the loyalty flag

In [ ]:
#ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
#ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
#ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
#checking basic stats for product prices based on each loyalty category
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

In [ ]:
# as you can see, loyal customers actually spend the least on average, with new customers spending the most.

In [12]:
#creating a new column with avg spent per user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\jphil\AppData\Local\Temp\ipykernel_26760\4113592259.py:2: FutureWarning: The provided callable <function mean at 0x0000023D62574F40> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [13]:
#creating a spending flag for users based on how much they usually spend
ords_prods_merge.loc[ords_prods_merge['avg_price'] <10, 'spending_flag'] = 'Low spender'

In [14]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >=10, 'spending_flag'] = 'High spender'

In [15]:
ords_prods_merge['spending_flag'].value_counts(dropna=False)

spending_flag
Low spender     31800381
High spender      634678
Name: count, dtype: int64

In [16]:
#creating a new column for median time between orders
ords_prods_merge['median_days']= ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\jphil\AppData\Local\Temp\ipykernel_26760\3705120424.py:1: FutureWarning: The provided callable <function median at 0x0000023D626F0D60> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days']= ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [17]:
#creating a frequency flag
ords_prods_merge.loc[ords_prods_merge['median_days'] >20, 'frequency_flag']= 'Non-frequent customer'

In [18]:
ords_prods_merge.loc[(ords_prods_merge['median_days'] >10) & (ords_prods_merge['median_days'] <=20), 'frequency_flag']= 'Regular customer'

In [19]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <=10, 'frequency_flag']= 'Frequent customer'

In [20]:
ords_prods_merge['frequency_flag'].value_counts(dropna=False)

frequency_flag
Frequent customer        20554936
Regular customer          9176025
Non-frequent customer     2704098
Name: count, dtype: int64

In [21]:
ords_prods_merge.to_pickle(os.path.join(path, 'ords_prods_flagged.pkl'))